# ETL JSON

CHEATSHEET


## About data converting:

| Source | Typical Format | Notes |
|--------|---------------|-------|
| APIs | JSON, XML | Almost always hierarchical; needs flattening into DataFrame or table for analysis |
| Web scraping | HTML, JSON, CSV | Usually converted into DataFrames for processing |
| Logs | JSON, plain text, CSV | Parsed and structured into DataFrames or database tables |
| Files from partners | Excel, CSV, JSON, Parquet | Some may already be tabular (CSV/Excel/Parquet), some need transformation |
| Databases | SQL tables | Already structured; can be queried directly into DataFrame without conversion |

In [9]:
import json
import pandas as pd

INPUT_FILE = "gsr_pos_logs.json"
OUTPUT_CSV = "gsr_pos_logs_cleaned.csv"

In [26]:
with open("qsr_pos_logs.json", "r") as file:
    data = json.load(file)
    display(type(data))  # Check the type of data
    display(data[8])  # Display the 9th record to understand its structure

list

{'order_id': 120008,
 'store_id': 102,
 'transaction_datetime': '2025-06-02 23:08:42',
 'business_day': '2025-06-02',
 'daypart': 'Late Night',
 'service_mode': 'Delivery',
 'menu_item': 'Chicken Sandwich',
 'modifier': 'Extra Sauce',
 'quantity': 2,
 'unit_price': 5.22,
 'discount': 0.0,
 'tax': 0.79,
 'total_amount': 11.73,
 'payment_type': 'Card'}

## from AI: full mini-ETL to process json

In [12]:
def extract(file_path):
    """Read JSON file and return Python data structure"""
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

def transform(data):
    """Convert JSON to DataFrame and clean/flatten"""
    # Flatten nested JSON if needed
    df = pd.json_normalize(data)
    
    # Example transformation: convert datetime column
    if 'transaction_datetime' in df.columns:
        df['transaction_datetime'] = pd.to_datetime(df['transaction_datetime'])
    
    # You can add more transformations here (e.g., renaming columns, filtering)
    return df

def load(df, output_path_csv=None, output_path_excel=None):
    """Save DataFrame to CSV or Excel"""
    if output_path_csv:
        df.to_csv(output_path_csv, index=False)
    if output_path_excel:
        df.to_excel(output_path_excel, index=False)

def etl_pipeline(input_file, output_csv=None, output_excel=None):
    """Full ETL process"""
    data = extract(input_file)
    df = transform(data)
    load(df, output_csv, output_excel)
    print("ETL completed successfully!")
    return df

# Example usage
if __name__ == "__main__":
    df_result = etl_pipeline(
        input_file="qsr_pos_logs.json",
        output_csv="qsr_pos_logs_cleaned.csv"
    )
display(df_result.head())


ETL completed successfully!


,order_id,store_id,transaction_datetime,business_day,daypart,service_mode,menu_item,modifier,quantity,unit_price,discount,tax,total_amount,payment_type
0,120000,102,2025-07-05 06:14:08,2025-07-05,Breakfast,Drive-Thru,Bottled Water,None,1,1.48,0.15,0.10,1.43,Cash
1,120001,201,2025-07-05 11:48:51,2025-07-05,Lunch,Dine-In,Chicken Sandwich,None,1,5.34,0.00,0.45,5.79,Card
2,120002,403,2025-04-13 08:05:35,2025-04-13,Breakfast,Takeout,Bottled Water,Light Ice,1,1.49,0.15,0.10,1.44,Cash
3,120003,301,2025-06-25 06:42:17,2025-06-25,Breakfast,Drive-Thru,Egg Muffin,No Cheese,1,3.62,0.00,0.34,3.96,Cash
4,120004,301,2025-08-01 12:58:36,2025-08-01,Lunch,Drive-Thru,Spicy Chicken Combo,None,2,9.17,0.00,1.38,19.72,Card


# Exploratory Data Analysis (EDA) Cheat Sheet (Basic)

## 1. Basic Info
```python
df.shape                           # number of rows and columns
df.columns                         # list column names
df.info()                          # data types and non-null counts
df.head()                          # first few rows
df.tail()                          # last few rows
df.count()                         # non-null values per column
df['col'].count()                  # non-null values in a specific column
df.dtypes                          # check data types
df['col'] = df['col'].astype(float)# convert column type



In [85]:
display(len(df_result))      # Check total number of rows
display(df_result.shape)     # Check the number of columns
print(df_result.columns)     # Check column names

display(df_result.shape[1])
display(df_result.shape[0])   # only 0=show number of rows and 1=shows number of columns
display(df_result.info())     # Check data types and non-null counts
display(round(df_result.describe(include='all'), 1))   # Get summary statistics for all columns, rounded to 1 decimal place


1743

(1743, 14)

Index(['order_id', 'store_id', 'transaction_datetime', 'business_day',
       'daypart', 'service_mode', 'menu_item', 'modifier', 'quantity',
       'unit_price', 'discount', 'tax', 'total_amount', 'payment_type'],
      dtype='object')


14

1743

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1743 entries, 0 to 1742
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   order_id              1743 non-null   int64         
 1   store_id              1743 non-null   int64         
 2   transaction_datetime  1743 non-null   datetime64[ns]
 3   business_day          1743 non-null   object        
 4   daypart               1743 non-null   object        
 5   service_mode          1727 non-null   object        
 6   menu_item             1743 non-null   object        
 7   modifier              657 non-null    object        
 8   quantity              1743 non-null   int64         
 9   unit_price            1743 non-null   float64       
 10  discount              1704 non-null   float64       
 11  tax                   1726 non-null   float64       
 12  total_amount          1732 non-null   float64       
 13  payment_type      

None

,order_id,store_id,transaction_datetime,business_day,daypart,service_mode,menu_item,modifier,quantity,unit_price,discount,tax,total_amount,payment_type
count,1743.0,1743.0,1743,1743,1743,1727,1743,657,1743.0,1743.0,1704.0,1726.0,1732.0,1702
unique,NaN,NaN,NaN,121,6,4,30,24,NaN,NaN,NaN,NaN,NaN,4
top,NaN,NaN,NaN,2025-07-16,Lunch,Dine-In,Chicken Sandwich Combo,Extra Sauce,NaN,NaN,NaN,NaN,NaN,Card
freq,NaN,NaN,NaN,24,508,527,129,56,NaN,NaN,NaN,NaN,NaN,962
mean,120871.0,266.9,2025-06-10 08:01:35.077452800,NaN,NaN,NaN,NaN,NaN,1.3,4.9,0.1,0.5,6.7,NaN
min,120000.0,101.0,2025-04-11 08:32:29,NaN,NaN,NaN,NaN,NaN,1.0,1.3,0.0,0.1,1.3,NaN
25%,120435.5,103.0,2025-05-11 10:49:24,NaN,NaN,NaN,NaN,NaN,1.0,2.8,0.0,0.2,3.5,NaN
50%,120871.0,202.0,2025-06-10 11:31:55,NaN,NaN,NaN,NaN,NaN,1.0,4.4,0.0,0.4,5.6,NaN
75%,121306.5,402.0,2025-07-09 16:23:02,NaN,NaN,NaN,NaN,NaN,1.0,6.2,0.0,0.6,9.3,NaN
max,121742.0,501.0,2025-08-09 20:49:34,NaN,NaN,NaN,NaN,NaN,4.0,11.2,3.9,2.5,32.7,NaN


### 2.Unique - missing values - duplicates

```python
df['col'].value_counts()   # frequency of each value
df['col'].nunique()        # number of unique values
df['col'].unique()         # list of unique values
df.isna().sum()            # missing values per column
df.isna().mean()           # percentage of missing values
df.duplicated().sum()          # number of duplicate rows
df.drop_duplicates(inplace=True) # remove duplicates


In [88]:
display(df_result.nunique())                            # Check number of unique values in each column
display(df_result.isnull().sum())                          # Check for missing values in each column
display(df_result.duplicated().sum())                      # Check for duplicate rows
display(df_result['modifier'].count())                     # Count non-null entries in 'modifier' column
#display(df_result['modifier'].isna())                     # Boolean Series indicating NaN values in 'modifier' column
display(df_result['modifier'].isna().sum())                # Count of NaN alues in 'modifier' column
display(df_result['modifier'].value_counts(dropna=False, ascending=False))  # Frequency of each unique value in 'modifier' column, including NaN
display(df_result['modifier'].value_counts(normalize=True, dropna=False).round(3))  # Relative frequency of each unique value in 'modifier' column, including NaN, normalized to proportions (%) and rounded(4)"""


order_id                1743
store_id                  10
transaction_datetime    1743
business_day             121
daypart                    6
service_mode               4
menu_item                 30
modifier                  24
quantity                   4
unit_price               676
discount                  96
tax                      156
total_amount             881
payment_type               4
dtype: int64

order_id                   0
store_id                   0
transaction_datetime       0
business_day               0
daypart                    0
service_mode              16
menu_item                  0
modifier                1086
quantity                   0
unit_price                 0
discount                  39
tax                       17
total_amount              11
payment_type              41
dtype: int64

np.int64(0)

np.int64(657)

np.int64(1086)

modifier
None                 1086
Extra Sauce            56
Cinnamon               53
No Sauce               41
Sugar-Free Syrup       39
No Cheese              39
No Pickles             34
Extra Ice              33
Light Ice              32
Almond Milk            29
Extra Egg              28
Small Size             27
Sausage Patty          26
Add Jalapenos          25
Add Bacon              25
No Ice                 24
Gluten-Free Bun        23
Spicy                  22
No Onions              20
Large Size             20
Oat Milk               17
Extra Cheese           17
Extra Patty            13
Caramel Topping         9
Chocolate Topping       5
Name: count, dtype: int64

modifier
None                 0.623
Extra Sauce          0.032
Cinnamon             0.030
No Sauce             0.024
Sugar-Free Syrup     0.022
No Cheese            0.022
No Pickles           0.020
Extra Ice            0.019
Light Ice            0.018
Almond Milk          0.017
Extra Egg            0.016
Small Size           0.015
Sausage Patty        0.015
Add Jalapenos        0.014
Add Bacon            0.014
No Ice               0.014
Gluten-Free Bun      0.013
Spicy                0.013
No Onions            0.011
Large Size           0.011
Oat Milk             0.010
Extra Cheese         0.010
Extra Patty          0.007
Caramel Topping      0.005
Chocolate Topping    0.003
Name: proportion, dtype: float64

### 3.Statistics

```python
df.describe()             # mean, std, min, max, quartiles
df['col'].mean()          # mean of a column
df['col'].median()        # median
df['col'].mode()          # most frequent value(s)
df['col'].var()           # variance
df['col'].std()           # standard deviation
df['col'].skew()          # skewness (asymmetry)
df['col'].kurtosis()      # kurtosis (tailedness)

In [87]:
display(round(df_result['total_amount'].min(), 2), 
        round(df_result['total_amount'].max(), 2), 
        round(df_result['total_amount'].mean(), 2), 
        round(df_result['total_amount'].std(), 2),        # Min, max, and mean of 'total_amount' column
        (df_result['total_amount'].mode()))               # count how many modes there are in 'total_amount' column (can be more than one mode) 
display(df_result['total_amount'].median())               # Median of 'total_amount' column, means: middle value when data is sorted
display(df_result['total_amount'].var().round(2))        # Variance of 'total_amount' column, means: how far a set of numbers is spread out from their average value

"""display(df_result['total_amount'].quantile([0.25, 0.5, 0.75]).round(2))  # 25th, 50th, and 75th percentiles of 'total_amount' column

display(df_result['total_amount'].skew().round(2))        # Skewness (asymmetry) of 'total_amount' column data, means: negative=left skewed, positive=right skewed, means: 
                                                          # if skewness is close to 0, data is symmetric, otherwise data is skewed: > 0: right skewed (bigger values), < 0: left skewed (smaller values)
display(df_result['total_amount'].kurtosis().round(2))    # Kurtosis: "tailedness" of 'total_amount' column data, means: high kurtosis (>3) = heavy tails, low kurtosis (<3) = light tails
                                                          # Normal distribution has kurtosis of 3.0. If kurtosis is >3, distribution has heavier tails and sharper peak than normal distribution. If kurtosis is <3, distribution has lighter tails and flatter peak than normal distribution.
                                                          # Kurtosis helps to understand the extremity of outliers in the data distribution."""

np.float64(1.32)

np.float64(32.69)

np.float64(6.71)

np.float64(4.58)

0    1.89
1    2.97
Name: total_amount, dtype: float64

np.float64(5.64)

np.float64(20.95)

'display(df_result[\'total_amount\'].quantile([0.25, 0.5, 0.75]).round(2))  # 25th, 50th, and 75th percentiles of \'total_amount\' column\n\ndisplay(df_result[\'total_amount\'].skew().round(2))        # Skewness (asymmetry) of \'total_amount\' column data, means: negative=left skewed, positive=right skewed, means: \n                                                          # if skewness is close to 0, data is symmetric, otherwise data is skewed: > 0: right skewed (bigger values), < 0: left skewed (smaller values)\ndisplay(df_result[\'total_amount\'].kurtosis().round(2))    # Kurtosis: "tailedness" of \'total_amount\' column data, means: high kurtosis (>3) = heavy tails, low kurtosis (<3) = light tails\n                                                          # Normal distribution has kurtosis of 3.0. If kurtosis is >3, distribution has heavier tails and sharper peak than normal distribution. If kurtosis is <3, distribution has lighter tails and flatter peak than normal distribution.\n

In [97]:
# Only include numerical columns for correlation
display(df_result.select_dtypes(include=['number']).corr())  # correlation matrix for numerical columns

# Example groupby: mean quantity per total_amount
display(df_result.groupby('total_amount')['quantity'].mean().round(1))  # Average quantity sold per total_amount value, rounded to 1 decimal place

,order_id,store_id,quantity,unit_price,discount,tax,total_amount
order_id,1.000000,0.023760,0.037516,0.012403,0.047185,0.007929,0.013449
store_id,0.023760,1.000000,0.007907,0.027566,0.023322,0.027272,0.027110
quantity,0.037516,0.007907,1.000000,-0.081007,0.053633,0.511906,0.536488
unit_price,0.012403,0.027566,-0.081007,1.000000,0.098101,0.705117,0.719485
discount,0.047185,0.023322,0.053633,0.098101,1.000000,0.052331,0.051486
tax,0.007929,0.027272,0.511906,0.705117,0.052331,1.000000,0.973782
total_amount,0.013449,0.027110,0.536488,0.719485,0.051486,0.973782,1.000000


total_amount
1.32     1.0
1.34     1.0
1.37     1.0
1.43     1.0
1.44     1.0
        ... 
30.69    3.0
30.82    3.0
31.77    3.0
32.32    3.0
32.69    3.0
Name: quantity, Length: 881, dtype: float64